# Wokflow con ZlightGBM

Para la familia de experimentos ZLIGHTGBM
del experimento colaborativo de Silvana Contreras y Sofía Scaiano.

Versión con modificaciones, basada en la notebook creada por Gustavo Denicolay.
Noviembre 2025.

## Inicializacion

In [ ]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
PARAM <- list()
PARAM$experimento <- "wfz_experimento_generico"
PARAM$semilla_primigenia <- 999199

In [ ]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

# Preprocesamiento

### Generacion de la clase_ternaria

In [ ]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz" )

#dataset <- fread("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [ ]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

### Feature Engineering Historico

In [ ]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

In [ ]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [ ]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

### lags deltas

no laguear las continuas cuando haya rankings

In [ ]:
# Creacion de LAGs (excluí monetarias originales)
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy(setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria"
   # , variables_continuas
    )
))

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Sys.time()

In [ ]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [ ]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

In [ ]:
ncol(dataset)
colnames(dataset)

# Produccion

In [ ]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

### Final Training Strategy

In [ ]:
PARAM$train_final$future <- c(202108)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912
, 202001, 202002, 202003
, 202004, 202005, 202006, 202007, 202008, 202009, 202010
, 202011, 202012
, 202101, 202102, 202103, 202104
#, 202105, 202106
)

PARAM$train_final$undersampling <- 0.05 

In [ ]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

#### Registros cambio las proporciones de POS/NEG

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

## Canaritos

In [ ]:

cat("Columnas ANTES:", ncol(dataset_train_final), "\n")

PARAM$qcanaritos <- 100L  #MODIFICAR SEGÚN EL EXPERIMENTO
cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)
for(i in seq(PARAM$qcanaritos)) {
  dataset_train_final[, paste0("canarito_", i) := runif(filas)]
}
cols_canaritos <- setdiff(colnames(dataset_train_final), cols0)
setcolorder(dataset_train_final, c(cols_canaritos, cols0))


cat("Canaritos creados:", length(cols_canaritos), "\n")
cat("Columnas DESPUÉS:", ncol(dataset_train_final), "\n")


### Final Models

In [ ]:
# dejo los datos en formato LightGBM

campos_buenos <- setdiff(colnames(dataset_train_final), c("clase_ternaria","clase01","training"))

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

In [ ]:
PARAM$lgbm <- list(
  boosting          = "gbdt",
  objective         = "binary",
  metric            = "custom",
  first_metric_only = FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise    = TRUE,
  verbosity         = -100,
  seed              = PARAM$semilla_primigenia,
  max_bin           = 31L,
  min_data_in_leaf  = 20L,
  num_iterations    = 9999L,
  num_leaves        = 9999L,
  learning_rate     = 1.0,
  feature_fraction  = 0.50,
  canaritos         = PARAM$qcanaritos,
  gradient_bound    = 0.05   # MODIFICAR SEGÚN EL EXPERIMENTO
)


In [ ]:
# Para compatibilidad con el bucle de semillerio de workflowtest:
PARAM$train_final$param_mejores <- PARAM$lgbm

In [ ]:
# Semillerio Final

library(primes)
PARAM$train_final$ksemillerio <- 1L  #MODIFICAR SEGÚN EL EXPERIMENTO
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq(PARAM$train_final$ksemillerio)]

## modelitos

In [ ]:
# FINAL TRAIN

dir.create("modelitos", showWarnings = FALSE)

param_completo <- copy(PARAM$train_final$param_mejores)

for (sem in PARAM$train_final$semillas) {
  arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, sem)

  if (!file.exists(arch_modelo)) {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data  = dtrain_final,
      param = param_completo
    )

    lgb.save(modelito, filename = arch_modelo)
    rm(modelito); gc()
    cat(" Modelo guardado seed", sem, "\n")
  } else {
    cat(" Ya existía, salto seed", sem, "\n")
  }
}
cat("\nListo. Si no había pendientes, no se recalculó nada.\n")


### Scoring

Se hace el predict() del modelo en los datos del futuro

In [ ]:
dfuture <- dataset[foto_mes %in% PARAM$train_final$future]

cat("Columnas ANTES:", ncol(dfuture), "\n")

######################

cols_fut_0 <- copy(colnames(dfuture))

filas_fut <- nrow(dfuture)
for(i in seq(PARAM$qcanaritos)) {
  dfuture[, paste0("canarito_", i) := runif(filas_fut)] 
}

cols_canaritos_fut <- setdiff(colnames(dfuture), cols_fut_0)
setcolorder(dfuture, c(cols_canaritos_fut, cols_fut_0))

cat("Canaritos creados:", length(cols_canaritos_fut), "\n")
cat("Columnas DESPUÉS:", ncol(dfuture), "\n")

#########################

mfuture <- data.matrix(dfuture[, campos_buenos, with = FALSE])

vpred_acum <- rep(0.0, nrow(dfuture))
qacumulados <- 0L

for(sem in PARAM$train_final$semillas) {
  arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, sem)
  modelo_final <- lgb.load(arch_modelo)
  vpred_individual <- predict(modelo_final, mfuture)

  tb_pred_seed <- dfuture[, .(numero_de_cliente, foto_mes)]
  tb_pred_seed[, prob := vpred_individual ]
  fwrite(tb_pred_seed,
         file = sprintf("prediccion_future_%s_%s.txt", PARAM$experimento, sem),
         sep  = "\t")

  vpred_acum <- vpred_acum + vpred_individual
  qacumulados <- qacumulados + 1L
}

vpred_acum <- vpred_acum / qacumulados

tb_prediccion <- dfuture[, .(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := vpred_acum ]
fwrite(tb_prediccion,
       file = sprintf("prediccion_future_%s.txt", PARAM$experimento),
       sep  = "\t")

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [ ]:
# genero archivos con los "envios" mejores
dir.create("kaggle", showWarnings = FALSE)

PARAM$cortes <- seq(7000, 16000, by = 500)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

envios <- 11000
tb_prediccion[, Predicted := 0L]           # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L]   # marco los primeros

archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(
  tb_prediccion[, .(numero_de_cliente, Predicted)],
  file = archivo_kaggle,
  sep = ","
)


In [ ]:
# genero archivos con los "envios" mejores
dir.create("kaggle", showWarnings = FALSE)

PARAM$cortes <- seq(7000, 16000, by = 500)

# ordeno por probabilidad descendente (no reordeno más de una vez)
tb_ord <- copy(tb_prediccion)
setorder(tb_ord, -prob)

# genero un archivo por cada corte en PARAM$cortes
for (envios in PARAM$cortes) {
  tb_ord[, Predicted := 0L]                          # seteo inicial a 0
  tb_ord[1:min(envios, .N), Predicted := 1L]         # marco los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_ord[, .(numero_de_cliente, Predicted)],
        file = archivo_kaggle,
        sep = ",",
        col.names = TRUE)

  cat("Generado:", basename(archivo_kaggle), "\n")
}


In [ ]:
Sys.time()

In [ ]:
cat("\n=== Guardando parámetros del experimento ===\n")
if (!requireNamespace("yaml", quietly = TRUE)) install.packages("yaml")

PARAM_export <- list(
  experimento  = PARAM$experimento,
  train_final  = list(
    future        = PARAM$train_final$future,
    training      = PARAM$train_final$training,
    undersampling = PARAM$train_final$undersampling,
    semillas      = PARAM$train_final$semillas,
    param_mejores = PARAM$train_final$param_mejores
  ),
  cortes         = if ("cortes" %in% names(PARAM)) PARAM$cortes else NULL,
  qcanaritos     = if ("qcanaritos" %in% names(PARAM)) PARAM$qcanaritos else NULL,
  campos_buenos  = if (exists("campos_buenos")) campos_buenos else NULL
)

nombre_archivo_param <- sprintf("PARAM_plano_exp_%s.yml", PARAM$experimento)
yaml::write_yaml(PARAM_export, file = nombre_archivo_param)
cat("Archivo de parámetros guardado exitosamente en:", nombre_archivo_param, "\n")



# TEST

### funciones

In [ ]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

PARAM$semilla_kaggle <- 314159

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle 
  )

  return( drealidad )
}

# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}


### predicciones test 

In [ ]:
#`dataset`, `PARAM`, `campos_buenos` y `realidad_evaluar` cargados

PARAM$false_future <- c(202106) 
d_false_future <- dataset[foto_mes %in% PARAM$false_future]

cat("Columnas ANTES:", ncol(d_false_future), "\n")

############3
cols_false_fut_0 <- copy(colnames(d_false_future))

filas_false_fut <- nrow(d_false_future)
for(i in seq(PARAM$qcanaritos)) {
  d_false_future[, paste0("canarito_", i) := runif(filas_false_fut)] 
}

cols_canaritos_false_fut <- setdiff(colnames(d_false_future), cols_false_fut_0)
setcolorder(d_false_future, c(cols_canaritos_false_fut, cols_false_fut_0))

cat("Canaritos creados:", length(cols_canaritos_false_fut), "\n")
cat("Columnas DESPUÉS:", ncol(d_false_future), "\n")
############


m_false_future <- data.matrix(d_false_future[, campos_buenos, with = FALSE])

# Inicializar la tabla de realidad 
drealidad <- realidad_inicializar(d_false_future, PARAM) 


# crear y guardar Predicciones del Ensemble y por Semilla 
cat("\n=== Generando predicciones del Ensemble y por Semilla ===\n")

vpred_acum_false <- rep(0.0, nrow(d_false_future))
qacumulados_false <- 0
todas_las_predicciones_semilla <- list() 

for (semilla in PARAM$train_final$semillas) {
    arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, semilla)
    if (file.exists(arch_modelo)) {
        modelo_individual <- lgb.load(arch_modelo) # Cargar modelo
        vpred_individual <- predict(modelo_individual, m_false_future) # Predecir

        # Acumular para el Ensemble
        vpred_acum_false <- vpred_acum_false + vpred_individual
        qacumulados_false <- qacumulados_false + 1

        # Crear y guardar predicción individual
        tb_prediccion_semilla <- d_false_future[, list(numero_de_cliente, foto_mes)]
        tb_prediccion_semilla[, prob := vpred_individual]
        nombre_archivo_salida_semilla <- paste0("prediccion_false_future_seed_", PARAM$experimento, "_", semilla, ".txt")
        fwrite(tb_prediccion_semilla, file = nombre_archivo_salida_semilla, sep = "\t")
        cat(" -> Semilla", semilla, "predicción guardada en:", nombre_archivo_salida_semilla, "\n")
        
        # Almacenar para uso posterior (curvas individuales)
        todas_las_predicciones_semilla[[as.character(semilla)]] <- tb_prediccion_semilla
        
    } else {
        cat(" -> ADVERTENCIA: Modelo no encontrado para la semilla:", semilla, "\n")
    }
}

# Calcular y guardar predicción del Ensemble Promedio
if (qacumulados_false > 0) {
    vpred_acum_false <- vpred_acum_false / qacumulados_false # Pasar a probabilidad
    tb_prediccion_false_future_prom <- d_false_future[, list(numero_de_cliente, foto_mes)]
    tb_prediccion_false_future_prom[, prob := vpred_acum_false]

    fwrite(tb_prediccion_false_future_prom,
           file = paste0("prediccion_false_future_ensemble_", PARAM$experimento, ".txt"), 
           sep = "\t"
    )
    cat("\n Predicción del Ensemble Promedio guardada en: prediccion_false_future_ensemble_", PARAM$experimento, ".txt\n")
} else {
    stop("No se encontraron modelos para generar predicciones del ensemble.")
}





In [ ]:
sys_time()

In [ ]:
# GANANCIASEN TEST  (semillas + ensemble + derivado )
cat("\n=== Calculando Ganancias de TODAS las Semillas en TODOS los Cortes ===\n")

ganancias_completas <- data.table()
PARAM$cortes <- seq(7000, 16000, by = 500)


for (semilla_name in names(todas_las_predicciones_semilla)) {

  tb_pred <- todas_las_predicciones_semilla[[semilla_name]]
  setorder(tb_pred, -prob)

  for (corte in PARAM$cortes) {
    tb_temp <- copy(tb_pred)
    tb_temp[, Predicted := 0L]
    tb_temp[1:corte, Predicted := 1L]

    res <- realidad_evaluar(drealidad, tb_temp)

    ganancias_completas <- rbind(
      ganancias_completas,
      data.table(
        tipo = paste0("Seed ", semilla_name),
        corte = corte,
        ganancia = res$total
      )
    )
  }

  cat("Semilla", semilla_name, "completada\n")
}


cat("Calculando ganancias para el Ensemble promedio\n")
setorder(tb_prediccion_false_future_prom, -prob)
tb_ensemble <- copy(tb_prediccion_false_future_prom)
mejor_ganancia_ensemble <- -Inf
mejor_corte <- NA

for (corte in PARAM$cortes) {
  tb_ensemble[, Predicted := 0L]
  tb_ensemble[1:corte, Predicted := 1L]

  res <- realidad_evaluar(drealidad, tb_ensemble)

 
  ganancias_completas <- rbind(
    ganancias_completas,
    data.table(
      tipo = "Ensemble",
      corte = corte,
      ganancia = res$total
    )
  )


  cat("Envios=", corte, "\t",
      " TOTAL=",   format(res$total,   big.mark = ","),
      " Public=",  format(res$public,  big.mark = ","),
      " Private=", format(res$private, big.mark = ","),
      "\n", sep = "")

  if (res$total > mejor_ganancia_ensemble) {
    mejor_ganancia_ensemble <- res$total
    mejor_corte <- corte
  }
}
cat("Corte óptimo del Ensemble (para línea roja):", mejor_corte, "envíos\n")


archivo_completo <- paste0("ganancias_todas_semillas_cortes_", PARAM$experimento, ".txt")
fwrite(ganancias_completas, file = archivo_completo, sep = "\t")
cat("\nGanancias completas (por corte) guardadas en:", archivo_completo, "\n")


resultados_ganancia_ensemble <- ganancias_completas[
  tipo == "Ensemble",
  .(envios = corte, ganancia_total = ganancia)
][order(envios)]

nombre_archivo_ganancias_ensemble <- paste0("ganancias_ensemble_exp_", PARAM$experimento, ".txt")
fwrite(resultados_ganancia_ensemble, file = nombre_archivo_ganancias_ensemble, sep = "\t")
cat("Ganancias del Ensemble (derivadas) guardadas en:", nombre_archivo_ganancias_ensemble, "\n")



In [ ]:
library(ggplot2)
library(scales)

semillas_plot <- ganancias_completas[tipo != "Ensemble"]
ensemble_plot <- ganancias_completas[tipo == "Ensemble"]


p_lineas <- ggplot(semillas_plot, aes(x = corte, y = ganancia, color = tipo, group = tipo)) +
    
    geom_line(size = 0.8, alpha = 0.7) +
    
    geom_line(data = ensemble_plot, 
          aes(x = corte, y = ganancia, color = "Ensemble"),
          size = 1.2, alpha = 1) +
    scale_color_discrete(breaks = c("Ensemble", unique(semillas_plot$tipo)))+


    geom_vline(xintercept = mejor_corte, linetype = "dashed", color = "red", size = 1) +
    
    # Escalas y Etiquetas
    scale_y_continuous(labels = scales::comma) +
    scale_x_continuous(labels = scales::comma) +
    labs(
        title = paste("Curvas de Ganancia por Semilla y Ensemble - EXP", PARAM$experimento),
        subtitle = paste("Línea roja: corte óptimo del Ensemble =", mejor_corte),
        x = "Corte (cantidad de envíos)",
        y = "Ganancia",
        color = "Modelo"
    ) +
    theme_minimal() +
    scale_color_discrete(breaks = c("Ensemble", unique(semillas_plot$tipo)))


print(p_lineas)


nombre_archivo_png <- paste0("curva_ganancia_semilla_corte_", PARAM$experimento, ".png")

ggsave(nombre_archivo_png, p_lineas, width = 14, height = 8, dpi = 300) 

cat("Gráfico de Ganancia por Corte guardado en:", nombre_archivo_png, "\n")

### grafico acumulado ensamble

In [ ]:
# Ordenar por probabilidad descendente
setorder(tb_prediccion_false_future_prom, -prob)

# Calcular ganancia acumulada
tb_prediccion_false_future_prom[, indice := 1:.N]

# Unir con drealidad para tener clase_ternaria
tb_ganancia <- merge(
  tb_prediccion_false_future_prom[, .(numero_de_cliente, foto_mes, prob, indice)],
  drealidad[, .(numero_de_cliente, foto_mes, clase_ternaria)],
  by = c("numero_de_cliente", "foto_mes")
)

setorder(tb_ganancia, indice)

tb_ganancia[, ganancia_individual := ifelse(clase_ternaria == "BAJA+2", 780000, -20000)]
tb_ganancia[, ganancia_acumulada := cumsum(ganancia_individual)]

ganancia_maxima <- max(tb_ganancia$ganancia_acumulada)
indice_maximo <- tb_ganancia[ganancia_acumulada == ganancia_maxima, indice][1]

umbral_ganancia <- ganancia_maxima * 0.66
tb_filtrada <- tb_ganancia[ganancia_acumulada >= umbral_ganancia]

p_acumulada <- ggplot(tb_filtrada, aes(x = indice, y = ganancia_acumulada)) +
  geom_line(color = "blue", linewidth = 1.2) +
  geom_point(data = tb_ganancia[indice == indice_maximo], 
             aes(x = indice, y = ganancia_acumulada),
             color = "red", size = 3) +
  annotate("text", 
           x = indice_maximo, 
           y = ganancia_maxima * 1.05,
           label = paste0("Ganancia Máxima\n", format(ganancia_maxima, big.mark = ",", scientific = FALSE)),
           color = "red", 
           fontface = "bold") +
  annotate("segment",
           x = indice_maximo, y = ganancia_maxima,
           xend = indice_maximo, yend = ganancia_maxima * 1.04,
           arrow = arrow(length = unit(0.3, "cm")),
           color = "red") +
  scale_x_continuous(labels = scales::comma) +
  scale_y_continuous(labels = scales::comma) +
  labs(
    title = paste0("Ganancia acumulada por orden de predicción (filtrada) - EXP ", PARAM$experimento),
    x = "Clientes ordenados por probabilidad",
    y = "Ganancia Acumulada"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(face = "bold", size = 14),
    panel.grid.minor = element_line(linewidth = 0.3, linetype = "dotted")
  )

print(p_acumulada)

nombre_archivo_acum <- paste0("curva_ganancia_acumulada_exp_", PARAM$experimento, ".png")
ggsave(nombre_archivo_acum, p_acumulada, width = 14, height = 8, dpi = 300)

cat("Ganancia máxima:", format(ganancia_maxima, big.mark = ","), "\n")
cat("Corte ideal por cliente:", indice_maximo, "\n")

### grafico acumulado semillas

In [ ]:
calc_curva_acum_filtrada <- function(tb_pred, drealidad, umbral_ganancia) {

  tb_ganancia <- copy(tb_pred)
  setorder(tb_ganancia, -prob)
  

  tb_ganancia[, indice := 1:.N]
  
  tb_ganancia <- merge(
    tb_ganancia[, .(numero_de_cliente, foto_mes, prob, indice)],
    drealidad[, .(numero_de_cliente, foto_mes, clase_ternaria)],
    by = c("numero_de_cliente", "foto_mes")
  )
  
  setorder(tb_ganancia, indice)
  tb_ganancia[, ganancia_individual := ifelse(clase_ternaria == "BAJA+2", 780000, -20000)]
  tb_ganancia[, ganancia_acumulada := cumsum(ganancia_individual)]
  
  ganancia_maxima <- max(tb_ganancia$ganancia_acumulada)
  
  if (missing(umbral_ganancia)) {
      umbral_ganancia <- ganancia_maxima * 0.66
  }
  
  tb_filtrada <- tb_ganancia[ganancia_acumulada >= umbral_ganancia]
  
  return(list(
      data_full = tb_ganancia, 
      data_plot = tb_filtrada, 
      maxima = ganancia_maxima
  ))
}


cat("\n=== Calculando Curvas de Ganancia Acumulada ===\n")

res_ensemble <- calc_curva_acum_filtrada(tb_prediccion_false_future_prom, drealidad)
ganancia_maxima <- res_ensemble$maxima
indice_maximo <- res_ensemble$data_full[ganancia_acumulada == ganancia_maxima, indice][1]
umbral_ganancia_base <- ganancia_maxima * 0.66 

cat("Máximo de Ganancia del Ensemble:", format(ganancia_maxima, big.mark = ","), "en el envío:", indice_maximo, "\n")

lista_curvas_plot <- list()
lista_curvas_plot[[1]] <- cbind(tipo = "Ensemble", res_ensemble$data_plot)

for(s in names(todas_las_predicciones_semilla)) {
    res_semilla <- calc_curva_acum_filtrada(todas_las_predicciones_semilla[[s]], drealidad, umbral_ganancia = umbral_ganancia_base)
    lista_curvas_plot[[length(lista_curvas_plot) + 1]] <- cbind(tipo = paste0("Seed ", s), res_semilla$data_plot)
}

todas_las_curvas_plot <- rbindlist(lista_curvas_plot, use.names = TRUE, fill = TRUE)

datos_ensemble_full <- res_ensemble$data_full

In [ ]:
library(RColorBrewer) 

semillas_list <- unique(todas_las_curvas_plot$tipo)
semillas_list <- semillas_list[semillas_list != "Ensemble"] 
num_semillas <- length(semillas_list)

colores_semillas_paleta <- rep(brewer.pal(12, "Paired"), ceiling(num_semillas / 12))[1:num_semillas]

colores_final <- setNames(c("black", colores_semillas_paleta), c("Ensemble", semillas_list))
breaks_final <- c("Ensemble", semillas_list) 


p_acumulada_ensemble <- ggplot(
    data = todas_las_curvas_plot, 
    aes(x = indice, y = ganancia_acumulada, color = tipo, group = tipo)
) +
  
  geom_line(
    aes(linewidth = ifelse(tipo == "Ensemble", 1.5, 0.4), 
        alpha = ifelse(tipo == "Ensemble", 1.0, 0.7))
  ) +
  
  scale_color_manual(
    values = colores_final,
    breaks = breaks_final, 
    name = "Modelo"
  ) +
  
  scale_linewidth_identity() + 
  scale_alpha_identity() +
  
  geom_point(data = datos_ensemble_full[indice == indice_maximo], 
             aes(x = indice, y = ganancia_acumulada),
             color = "red", size = 4, inherit.aes = FALSE) + 
             
  annotate("label", 
           x = indice_maximo, 
           y = ganancia_maxima * 1.05,
           label = paste0("Máximo\n", format(ganancia_maxima, big.mark = ",", scientific = FALSE)),
           color = "red", 
           fontface = "bold",
           fill = "white",
           size = 4) +
           
  geom_segment(data = datos_ensemble_full[indice == indice_maximo], 
               aes(x = indice_maximo, y = ganancia_maxima,
                   xend = indice_maximo, yend = ganancia_maxima * 1.04),
               arrow = arrow(length = unit(0.3, "cm")),
               color = "red", 
               inherit.aes = FALSE) + 
  
  # Escalas y Etiquetas
  scale_x_continuous(labels = scales::comma, name = "Clientes ordenados por probabilidad (Índice / Envíos)") +
  scale_y_continuous(labels = scales::comma, name = "Ganancia Acumulada") +
  labs(
    title = paste0("Ganancia Acumulada (Semillas y Ensemble) con Filtrado - EXP ", PARAM$experimento)
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(face = "bold", size = 14),
    legend.position = "right",
    panel.grid.minor = element_line(color = "gray90") 
  )
print(p_acumulada_ensemble)

nombre_archivo_acum <- paste0("curva_ganancia_acumulada_ensemble_semillas__", PARAM$experimento, ".png")
ggsave(nombre_archivo_acum, p_acumulada_ensemble, width = 14, height = 8, dpi = 300)

cat("Gráfico de Ganancia Acumulada guardado en:", nombre_archivo_acum, "\n")

In [ ]:
Sys.time()